In [5]:
import numpy as np
import xarray as xr
from pathlib import Path
import os

import utilities as ut

In [8]:
data_info_dict = ut.json2dict('data_info.json')

def fetch(name, destination='./data', reload=False):
    destination = Path(destination)
    if not os.path.exists(destination):
        destination.mkdir(parents=True)

    if name.endswith('.nc'):
        name = name[:-3]

    file = destination / f'{name}.nc'
    if not os.path.exists(file) or reload:
        try:
            link = data_info_dict[name]['link']
        except KeyError:
            raise KeyError(f'No item {name} in data_info.json')
        os.system(f"wget -O {file} \"{link}\"")
    else:
        print('file already loaded')

In [9]:
fetch('mrso10')

--2022-04-11 11:43:51--  https://mycore.core-cloud.net/index.php/s/r9XRFyOQgmsAvNG/download
Risoluzione di mycore.core-cloud.net (mycore.core-cloud.net)... 64:ff9b::c239:8af7, 64:ff9b::c239:8af0, 194.57.138.240, ...
Connessione a mycore.core-cloud.net (mycore.core-cloud.net)|64:ff9b::c239:8af7|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 413481 (404K) [application/octet-stream]
Salvataggio in: "data/mrso10.nc"

     0K .......... .......... .......... .......... .......... 12% 3,48M 0s
    50K .......... .......... .......... .......... .......... 24% 1,14M 0s
   100K .......... .......... .......... .......... .......... 37% 2,06M 0s
   150K .......... .......... .......... .......... .......... 49% 4,73M 0s
   200K .......... .......... .......... .......... .......... 61% 3,77M 0s
   250K .......... .......... .......... .......... .......... 74% 4,25M 0s
   300K .......... .......... .......... .......... .......... 86% 4,06M 0s
   350K .....

In [11]:
m = xr.open_dataset('./data/mrso10.nc')
m

<xarray.Dataset>
Dimensions:    (time: 10, bnds: 2, lon: 128, lat: 64)
Coordinates:
  * time       (time) object 0001-05-01 10:30:00 ... 0001-05-10 10:30:00
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat        (lat) float64 87.86 85.1 82.31 79.53 ... -82.31 -85.1 -87.86
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    mrso       (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.6 (http://mpimet.mpg.de/...
    Conventions:  CF-1.0
    history:      Tue Feb 23 09:36:35 2021: cdo cat /scratch/gmiloshe/PLASIM/...
    title:        PUMA/PLASIM DATA
    frequency:    day
    CDO:          Climate Data Operators version 1.9.6 (http://mpimet.mpg.de/...

In [12]:
from cftime import Datetime360Day

In [19]:
d = Datetime360Day(1, 1, 1)
d

cftime.Datetime360Day(1, 1, 1, 0, 0, 0, 0, has_year_zero=True)

In [26]:
d = d.replace(year=d.year + 2)
d

cftime.Datetime360Day(4, 1, 1, 0, 0, 0, 0, has_year_zero=True)

In [29]:
@np.vectorize
def traslate(a, y):
    return a.replace(year = a.year + y)

In [30]:
m.time

<xarray.DataArray 'time' (time: 10)>
array([cftime.Datetime360Day(1, 5, 1, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 2, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 3, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 4, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 5, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 6, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 7, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 8, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 9, 10, 30, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(1, 5, 10, 10, 30, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 0001-05-01 10:30:00 ... 0001-05-10 10:30:00
Attributes:
    standard_name:  time
    bounds:         time_bnds
    axis:           T

In [43]:
new_time = []
new_time.append(traslate(m.time[:5], 0))
new_time.append(traslate(m.time[5:], 2))
new_time = np.concatenate(new_time)

m = m.assign_coords({'time': new_time})
m

<xarray.Dataset>
Dimensions:    (time: 10, bnds: 2, lon: 128, lat: 64)
Coordinates:
  * time       (time) object 0011-05-01 10:30:00 ... 0013-05-10 10:30:00
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat        (lat) float64 87.86 85.1 82.31 79.53 ... -82.31 -85.1 -87.86
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object 0001-05-01 00:00:00 ... 0001-05-10 21:00:00
    mrso       (time, lat, lon) float32 0.0 0.0 0.0 ... 0.0009993 0.001 0.001
Attributes:
    CDI:          Climate Data Interface version 1.9.6 (http://mpimet.mpg.de/...
    Conventions:  CF-1.0
    history:      Tue Feb 23 09:36:35 2021: cdo cat /scratch/gmiloshe/PLASIM/...
    title:        PUMA/PLASIM DATA
    frequency:    day
    CDO:          Climate Data Operators version 1.9.6 (http://mpimet.mpg.de/...

In [34]:
help(m.assign)

Help on method assign in module xarray.core.dataset:

assign(variables: Mapping[Any, Any] = None, **variables_kwargs: Hashable) -> 'Dataset' method of xarray.core.dataset.Dataset instance
    Assign new data variables to a Dataset, returning a new object
    with all the original variables in addition to the new ones.
    
    Parameters
    ----------
    variables : mapping of hashable to Any
        Mapping from variables names to the new values. If the new values
        are callable, they are computed on the Dataset and assigned to new
        data variables. If the values are not callable, (e.g. a DataArray,
        scalar, or array), they are simply assigned.
    **variables_kwargs
        The keyword arguments form of ``variables``.
        One of variables or variables_kwargs must be provided.
    
    Returns
    -------
    ds : Dataset
        A new Dataset with the new variables in addition to all the
        existing variables.
    
    Notes
    -----
    Since ``kwargs`